In [297]:
import csv
import gspread
import pandas as pd
import google.auth
import time
import gspread_dataframe as gd
from google.colab import drive
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from googleapiclient.discovery import build

auth.authenticate_user()
drive.mount("/drive")
creds, _ = default()
gc = gspread.authorize(creds)
overpass_url = "http://overpass-api.de/api/interpreter"

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)
creds, _ = google.auth.default()

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [298]:
# Adjust as needed
# The folder the results are saved to:
output_folder = "1WzGGh_Xv23YlrBWHgunrecoicbjSMEoS"
# The folder containing the subfolders for each size of datasets; the subfolders contain the results of each fold (find an example at xxx):
search_folder_id = "17vAQZmIDpAw39HL-F_ix_fWxN3-H3gdZ"
# The ID of the spreadsheet containing the calculation template
calc_templates_ssid = "1MyPUXrpzQbnITkJQ7ZSbKNRXyLyboF0Y9Lrbs9xGpS0"
# The name of the worksheet with the calculations for the F1 score (empty columns are ignored!) 
calc_template_name = "BertEnNameTemplates"
# Shows the position of the F1 Scores of each entity + average on the worksheets
cell_positions_of_averages = {
    "First_Name": "AH2",
    "Middle_Name": "AH3",
    "Last_Name": "AH4",
    "Average": "AH5"
}

In [299]:
# Generate the calculations for the F1 scores for each file type
templatesheet = gc.open_by_key(calc_templates_ssid)
template_worksheet = templatesheet.worksheet(calc_template_name)
calculation= gd.get_as_dataframe(template_worksheet, evaluate_formulas=False)
calculation.dropna(how='all', axis=1, inplace=True)
calculation.dropna(how='all', inplace=True)

In [300]:
def search_file(folder_id, mimetype):
    """Search file in drive location

    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.
    """
    try:
        # create drive api client
        service = build('drive', 'v3', credentials=creds)
        files = []
        page_token = None
        while True:
            # pylint: disable=maybe-no-member
            response = service.files().list(q = "'" + folder_id + "' in parents and mimeType = '" + mimetype + "'",
                                            spaces='drive',
                                            fields='nextPageToken, '
                                                   'files(id, name)',
                                            pageToken=page_token).execute()
            for file in response.get('files', []):
                # Process change
                print(F'Found: {file.get("name")}, {file.get("id")}')

            files.extend(response.get('files', []))
            page_token = response.get('nextPageToken', None)
            if page_token is None:
                break

    except HttpError as error:
        print(F'An error occurred: {error}')
        files = None

    return files


In [301]:
def create_spreadsheet(sheet_title, output_folder_id):
  """
  Create a new spreadsheet with the given title (sheet_title). Can be saved in a specific folder by giving the folder_id in a list
  """
  drive_api = build('drive', 'v3', credentials=creds)

  print("Creating Sheet %s", sheet_title)
  body = {
      'name': sheet_title,
      'mimeType': 'application/vnd.google-apps.spreadsheet',
  }

  if output_folder_id:
      body["parents"] =  output_folder_id

  req = drive_api.files().create(body=body)
  new_sheet = req.execute()
  
  return new_sheet["id"]

In [302]:
#https://stackoverflow.com/a/2556252/13590692
def rreplace(s, old, new, occurrence):
  """
  Replace last x occurences of a substring in a string with a new substring
  """
  li = s.rsplit(old, occurrence)
  return new.join(li)

In [303]:
def add_average_worksheet(spreadsheet, worksheet_names):
  """
  Generates the worksheet displaying the averages of each entity over the pages
  """
  entities = []
  formulas = []
  for entity in cell_positions_of_averages:
    entities.append(entity)
    formula = "=AVERAGE("
    for name in worksheet_names:
      formula = F"{formula}'{name}'!{cell_positions_of_averages[entity]};"
    formula = rreplace(formula, ';', '', 1)
    formulas.append(F"{formula})")
  
  worksheet_content = {"Entity": entities, "F1": formulas}
  
  worksheet = spreadsheet.add_worksheet(title="Average", rows=len(entities), cols = len(worksheet_content))
  gd.set_with_dataframe(worksheet, pd.DataFrame(data=worksheet_content))

In [304]:
def add_calculations(worksheet):
  """
  Add the calculations for the F1 score to a worksheet (based on a given template)
  """
  worksheet_df = pd.DataFrame(worksheet.get_all_records())

  for column in calculation:
    if column in worksheet_df:
      print("Worksheet has same columns as calculations, will not be updated for: " + row)
    else:
      worksheet_df[column] = calculation[column]

  if (len(worksheet_df) < len(calculation)):
    difference = len(worksheet_df) - len(calculation)
    for i in range(len(worksheet_df), len(calculation)):
      worksheet_df = worksheet_df.append(calculation.iloc[i], ignore_index=True)
  
  gd.set_with_dataframe(worksheet, worksheet_df)

In [305]:
def add_csv_files_to_spreadsheet(files, spreadsheet):
  # Generating a list of the sheet names saves API calls (less risk of exceeding the quota)
  worksheet_names = list()

  for file in files:
    filename = file.get("name")
    sheetName = filename.split(".")[0]
    sheet_id = file.get("id")
    downloaded_content = gdrive.CreateFile({'id' : sheet_id})
    downloaded_content.GetContentFile(filename) 
    content_df = pd.read_csv(filename)
    no_rows = len(content_df)
    no_cols = len(content_df.columns)
    
    worksheet = spreadsheet.add_worksheet(title=sheetName, rows = no_rows, cols = no_cols)
    gd.set_with_dataframe(worksheet, content_df)
    worksheet = add_calculations(worksheet)
    worksheet_names.append(sheetName)

  # Delete Fragment from creation
  spreadsheet.del_worksheet(spreadsheet.sheet1)

  # Add average worksgeet
  add_average_worksheet(spreadsheet, worksheet_names)
  
  return worksheet_names

In [306]:
def generate_spreadsheet_from_folder(input_folder_id):
  """
  Move all CSV files in a folder into one spreadsheet
  """
  files = search_file(input_folder_id, "text/csv")
  
  if len(files) <= 0:
    print("No csv files found")
    return

  spreadsheet_name = files[0].get("name").split("-")[0]
  parent_list = [output_folder]
  spread_id = create_spreadsheet(spreadsheet_name, parent_list)

  spreadsheet = gc.open_by_key(spread_id)

  worksheets = add_csv_files_to_spreadsheet(files, spreadsheet)

In [307]:
def iterate_over_folders(directory_id):
  """
  Find all folders in a given subdirectory and iterate over them to move the csv files of each folder in their separate spreadsheets.
  """
  folders = search_file(directory_id, "application/vnd.google-apps.folder")
  for folder in folders:
    generate_spreadsheet_from_folder(folder.get("id"))

    print("Initiating wait time of 20 seconds as to not exceeding the limit of requests.")
    time.sleep(20)

In [ ]:
iterate_over_folders(search_folder_id)

Found: 10, 129Kw66lwaRzd-Q2QzRcBBlXAa1ADGDw_
Found: 1000, 1Y-mF6iZ003KvDd74Slkx407K4lQ8sIBv
Found: 100, 1Q1MqauGNsgTRMVk1cgu3zP1LHExsrINA
Found: 250, 1J5FaUBF79LtOhbga857bTc0PVCbJcwha
Found: 50, 1rx-GrtP9c41aHz6Jr410cqiHxOSw5NBI
Found: 500, 1DnegjId99ZdjejR7RqjKzusMHAkG62C6
Found: short_name_templates10-4.csv, 1UglGkVJD-jeMbkJ8KE_EVMBZ4YPo1SwQ
Found: short_name_templates10-3.csv, 1EyqK6VXpVk5uuDMyUzjRNZI7Qg3ws3GM
Found: short_name_templates10-2.csv, 1mFdTPJNNTtTd3UF3-AoRFnCA_OSomwzY
Found: short_name_templates10-1.csv, 1vsxYSdWerI0bhc9IUlbXeWIha401GTFg
Found: short_name_templates10-0.csv, 11mmTk9eZTJaZAbQqsu1sVz_wjGa7_C8M
Creating Sheet %s short_name_templates10
Initiating wait time of 20 seconds as to not exceeding the limit of requests.
Found: short_name_templates1000-4.csv, 1hkK_LJfS91mD_EOj4deKTPILvEwRrGWT
Found: short_name_templates1000-3.csv, 1Cm0JtAiZqPui9Gtag0k7_Sl3udxAPKCa
Found: short_name_templates1000-2.csv, 1lY6tn9sS_bc4rfVBqm1w2ai8XvFInFkI
Found: short_name_templates1000-